# MOVIELENS 20 M - Data Processing and Analysis

### To Check if Spark is active

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### Importing the required libraries

In [2]:
import os
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import pandas as pd
spark = SparkSession.builder.master("local").config(conf=SparkConf()).getOrCreate()

### Converting the data into Spark DataFrames

In [3]:
movie_df=spark.read.csv('movie.csv',inferSchema=True,header=True)
link_df=spark.read.csv('link.csv',inferSchema=True,header=True)
rating_df=spark.read.csv('rating.csv',inferSchema=True,header=True)
tag_df=spark.read.csv('tag.csv',inferSchema=True,header=True)

In [4]:
# Movies dataframe
movie_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [5]:
# Link dataframe
link_df.show()

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
|      6|113277|   949|
|      7|114319| 11860|
|      8|112302| 45325|
|      9|114576|  9091|
|     10|113189|   710|
|     11|112346|  9087|
|     12|112896| 12110|
|     13|112453| 21032|
|     14|113987| 10858|
|     15|112760|  1408|
|     16|112641|   524|
|     17|114388|  4584|
|     18|113101|     5|
|     19|112281|  9273|
|     20|113845| 11517|
+-------+------+------+
only showing top 20 rows



In [6]:
#Tags Dataframe
tag_df.show()

+------+-------+-----------------+-------------------+
|userId|movieId|              tag|          timestamp|
+------+-------+-----------------+-------------------+
|    18|   4141|      Mark Waters|2009-04-24 18:19:40|
|    65|    208|        dark hero|2013-05-10 01:41:18|
|    65|    353|        dark hero|2013-05-10 01:41:19|
|    65|    521|    noir thriller|2013-05-10 01:39:43|
|    65|    592|        dark hero|2013-05-10 01:41:18|
|    65|    668|        bollywood|2013-05-10 01:37:56|
|    65|    898| screwball comedy|2013-05-10 01:42:40|
|    65|   1248|    noir thriller|2013-05-10 01:39:43|
|    65|   1391|             mars|2013-05-10 01:40:55|
|    65|   1617|         neo-noir|2013-05-10 01:43:37|
|    65|   1694|            jesus|2013-05-10 01:38:45|
|    65|   1783|    noir thriller|2013-05-10 01:39:43|
|    65|   2022|            jesus|2013-05-10 01:38:45|
|    65|   2193|           dragon|2013-05-10 02:01:54|
|    65|   2353|conspiracy theory|2013-05-10 02:01:06|
|    65|  

In [7]:
#Ratings Dataframe
rating_df.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
|     1|    112|   3.5|2004-09-10 03:09:00|
|     1|    151|   4.0|2004-09-10 03:08:54|
|     1|    223|   4.0|2005-04-02 23:46:13|
|     1|    253|   4.0|2005-04-02 23:35:40|
|     1|    260|   4.0|2005-04-02 23:33:46|
|     1|    293|   4.0|2005-04-02 23:31:43|
|     1|    296|   4.0|2005-04-02 23:32:47|
|     1|    318|   4.0|2005-04-02 23:33:18|
|     1|    337|   3.5|2004-09-10 03:08:29|
|     1|    367|   3.5|2005-04-02 23:53:00|
|     1|    541|   4.0|2005-04-02 23:30:03|
|     1|    589|   3.5|2005-04-02 23:45:57|
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|    653|   3.0|2004-09-10 03:08:11|
|     1|    919|   3.5|2004-09-1

### Correlation of the data

In [8]:
def correlation_matrix(df, corr_columns, method='pearson'):
    vector_col = "corr_features"
    assembler = VectorAssembler(inputCols=corr_columns, outputCol=vector_col)
    df_vector = assembler.transform(df).select(vector_col)
    matrix = Correlation.corr(df_vector, vector_col, method)

    result = matrix.collect()[0]["pearson({})".format(vector_col)].values
    return pd.DataFrame(result.reshape(-1, len(corr_columns)), columns=corr_columns, index=corr_columns)

In [9]:
# Correlation of ratings dataset
correlation_matrix(rating_df,['userId','movieId','rating'])

,userId,movieId,rating
userId,1.000000,-0.000850,0.001175
movieId,-0.000850,1.000000,0.002606
rating,0.001175,0.002606,1.000000


In [10]:
# Correlation of Tags dataset
correlation_matrix(tag_df,['userId','movieId'])

,userId,movieId
userId,1.000000,-0.006086
movieId,-0.006086,1.000000


### Extracting the year and the genres for all movies

In [11]:
movie_df.select("title","genres").show()

+--------------------+--------------------+
|               title|              genres|
+--------------------+--------------------+
|    Toy Story (1995)|Adventure|Animati...|
|      Jumanji (1995)|Adventure|Childre...|
|Grumpier Old Men ...|      Comedy|Romance|
|Waiting to Exhale...|Comedy|Drama|Romance|
|Father of the Bri...|              Comedy|
|         Heat (1995)|Action|Crime|Thri...|
|      Sabrina (1995)|      Comedy|Romance|
| Tom and Huck (1995)|  Adventure|Children|
| Sudden Death (1995)|              Action|
|    GoldenEye (1995)|Action|Adventure|...|
|American Presiden...|Comedy|Drama|Romance|
|Dracula: Dead and...|       Comedy|Horror|
|        Balto (1995)|Adventure|Animati...|
|        Nixon (1995)|               Drama|
|Cutthroat Island ...|Action|Adventure|...|
|       Casino (1995)|         Crime|Drama|
|Sense and Sensibi...|       Drama|Romance|
|   Four Rooms (1995)|              Comedy|
|Ace Ventura: When...|              Comedy|
|  Money Train (1995)|Action|Com

In [12]:
# Testing and normally extracting the year
movie_df_title_yr = movie_df.select("title").collect()
final_yr = []
for i in movie_df_title_yr:
    final_yr.append(i[0][-5:-1])
final_yr[0:10]


['1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995']

In [13]:
#Extracting the year in the title column and showing it with genre

from pyspark.sql.functions import regexp_replace,col
from pyspark.sql.functions import udf
import re

def years(s):
    return s[-5:-1]

yr_udf = udf(years)

movie_df.withColumn("title", yr_udf("title")).show()

+-------+-----+--------------------+
|movieId|title|              genres|
+-------+-----+--------------------+
|      1| 1995|Adventure|Animati...|
|      2| 1995|Adventure|Childre...|
|      3| 1995|      Comedy|Romance|
|      4| 1995|Comedy|Drama|Romance|
|      5| 1995|              Comedy|
|      6| 1995|Action|Crime|Thri...|
|      7| 1995|      Comedy|Romance|
|      8| 1995|  Adventure|Children|
|      9| 1995|              Action|
|     10| 1995|Action|Adventure|...|
|     11| 1995|Comedy|Drama|Romance|
|     12| 1995|       Comedy|Horror|
|     13| 1995|Adventure|Animati...|
|     14| 1995|               Drama|
|     15| 1995|Action|Adventure|...|
|     16| 1995|         Crime|Drama|
|     17| 1995|       Drama|Romance|
|     18| 1995|              Comedy|
|     19| 1995|              Comedy|
|     20| 1995|Action|Comedy|Cri...|
+-------+-----+--------------------+
only showing top 20 rows



# Spark-SQL Analysis

## Creating Views for spark dataframes

In [14]:
rating_df.createOrReplaceTempView("ratings")
movie_df.createOrReplaceTempView("movies")
tag_df.createOrReplaceTempView("tags")
link_df.createOrReplaceTempView("links")
# Movies with year separated from the title
movie_df.withColumn("title", yr_udf("title")).createOrReplaceTempView("movies_year")

### Query 1: How many number of Movies are there for each rating? 

In [15]:
spark.sql("SELECT rating,COUNT(movieID) as No_Of_Movies FROM ratings GROUP BY rating ORDER BY rating DESC").show()

+------+------------+
|rating|No_Of_Movies|
+------+------------+
|   5.0|     2898660|
|   4.5|     1534824|
|   4.0|     5561926|
|   3.5|     2200156|
|   3.0|     4291193|
|   2.5|      883398|
|   2.0|     1430997|
|   1.5|      279252|
|   1.0|      680732|
|   0.5|      239125|
+------+------------+



### Query 2: What are top 10 most rated movies?

In [16]:
query = "Select m.title,r.movieID,r.rating from ratings r, movies m where m.movieID = r.movieID order by r.rating desc limit 10"
spark.sql(query).show()

+--------------------+-------+------+
|               title|movieID|rating|
+--------------------+-------+------+
| Blade Runner (1982)|    541|   5.0|
|Amityville Horror...|   1327|   5.0|
|Terminator 2: Jud...|    589|   5.0|
|Jurassic Park (1993)|    480|   5.0|
|2001: A Space Ody...|    924|   5.0|
|Lord of the Rings...|   7153|   5.0|
|Star Wars: Episod...|   1196|   5.0|
|Mr. Holland's Opu...|     62|   5.0|
|Star Wars: Episod...|   1210|   5.0|
|Star Wars: Episod...|    260|   5.0|
+--------------------+-------+------+



### Query 3: How many users have rated each movie?

In [17]:
spark.sql("SELECT movieID, COUNT(userID) as No_Of_Users FROM ratings GROUP BY movieID ORDER BY COUNT(userID) DESC").show()

+-------+-----------+
|movieID|No_Of_Users|
+-------+-----------+
|    296|      67310|
|    356|      66172|
|    318|      63366|
|    593|      63299|
|    480|      59715|
|    260|      54502|
|    110|      53769|
|    589|      52244|
|   2571|      51334|
|    527|      50054|
|      1|      49695|
|    457|      49581|
|    150|      47777|
|    780|      47048|
|     50|      47006|
|   1210|      46839|
|    592|      46054|
|   1196|      45313|
|   2858|      44987|
|     32|      44980|
+-------+-----------+
only showing top 20 rows



### Query 4: What is the Total Rating for each movie? 

In [18]:
query = 'SELECT movieID, SUM(rating) as Total_Rating FROM ratings GROUP BY movieID ORDER BY movieID'
spark.sql(query).show()

+-------+------------+
|movieID|Total_Rating|
+-------+------------+
|      1|    194866.0|
|      2|     71444.0|
|      3|     40128.5|
|      4|      7886.0|
|      5|     37268.5|
|      6|     91651.0|
|      7|     43633.0|
|      8|      4446.0|
|      9|     11899.5|
|     10|     99488.0|
|     11|     66613.0|
|     12|     10073.0|
|     13|      4781.0|
|     14|     20668.0|
|     15|      7921.0|
|     16|     65879.0|
|     17|     82018.5|
|     18|     17553.0|
|     19|     54594.0|
|     20|     11765.0|
+-------+------------+
only showing top 20 rows



### Query 5: What is the Average Rating for each movie?

In [19]:
query = 'SELECT movieID, ROUND(AVG(rating),2) as Average_Rating FROM ratings GROUP BY movieID ORDER BY movieID'
spark.sql(query).show()

+-------+--------------+
|movieID|Average_Rating|
+-------+--------------+
|      1|          3.92|
|      2|          3.21|
|      3|          3.15|
|      4|          2.86|
|      5|          3.06|
|      6|          3.83|
|      7|          3.37|
|      8|          3.14|
|      9|           3.0|
|     10|          3.43|
|     11|          3.67|
|     12|          2.62|
|     13|          3.27|
|     14|          3.43|
|     15|          2.72|
|     16|          3.79|
|     17|          3.97|
|     18|          3.37|
|     19|          2.61|
|     20|          2.88|
+-------+--------------+
only showing top 20 rows



### Query 6: How many movies are there for each genre?

In [20]:
spark.sql("Select genres, count(movieID) as Total_Movies from movies group by genres order by count(movieID) desc").show()

+--------------------+------------+
|              genres|Total_Movies|
+--------------------+------------+
|               Drama|        4520|
|              Comedy|        2294|
|         Documentary|        1942|
|        Comedy|Drama|        1264|
|       Drama|Romance|        1075|
|      Comedy|Romance|         757|
|Comedy|Drama|Romance|         605|
|              Horror|         565|
|         Crime|Drama|         448|
|      Drama|Thriller|         426|
|           Drama|War|         399|
|     Horror|Thriller|         374|
|Crime|Drama|Thriller|         304|
|            Thriller|         268|
|  (no genres listed)|         246|
|             Western|         215|
|        Action|Drama|         210|
|        Comedy|Crime|         187|
|              Action|         178|
|       Comedy|Horror|         170|
+--------------------+------------+
only showing top 20 rows



### Query 7: How many movies have been rated each year?

In [21]:
spark.sql("Select title as Year, count(movieID) as Total_Movies from movies_year group by title order by count(movieID) desc").show()

+----+------------+
|Year|Total_Movies|
+----+------------+
|2009|        1113|
|2012|        1021|
|2011|        1014|
|2013|        1010|
|2008|         978|
|2010|         958|
|2007|         900|
|2006|         854|
|2014|         740|
|2005|         739|
|2004|         706|
|2002|         678|
|2003|         655|
|2001|         632|
|2000|         613|
|1998|         554|
|1999|         542|
|1997|         528|
|1996|         509|
|1995|         474|
+----+------------+
only showing top 20 rows



### Query 8: Which were the least rated movies in the year of 2005?

In [22]:
query = "Select distinct(m.title),r.rating,m1.title as Year from movies m, ratings r, movies_year as m1 where m.movieID = r.movieID and m.movieID = m1.movieID and r.movieID = m1.movieId and m1.title = '2005' order by r.rating"
spark.sql(query).show()

+--------------------+------+----+
|               title|rating|Year|
+--------------------+------+----+
|Today You Die (2005)|   0.5|2005|
|Strangers with Ca...|   0.5|2005|
|       Cursed (2005)|   0.5|2005|
|Leonard Cohen: I'...|   0.5|2005|
|Pacifier, The (2005)|   0.5|2005|
|Miss Congeniality...|   0.5|2005|
|Day of the Dead 2...|   0.5|2005|
|Proposition, The ...|   0.5|2005|
|Bad News Bears (2...|   0.5|2005|
|Time to Leave (2005)|   0.5|2005|
|       Hostel (2005)|   0.5|2005|
|  Match Point (2005)|   0.5|2005|
|Star Wreck: In th...|   0.5|2005|
|   Dirty Love (2005)|   0.5|2005|
|On a Clear Day (2...|   0.5|2005|
|God Who Wasn't Th...|   0.5|2005|
|  Constantine (2005)|   0.5|2005|
|Wayward Cloud, Th...|   0.5|2005|
|  Keeping Mum (2005)|   0.5|2005|
|Mr. & Mrs. Smith ...|   0.5|2005|
+--------------------+------+----+
only showing top 20 rows



### Query 9: What are the genres of the top 5 rated movies?

In [23]:
query = "Select m.title,m.genres,r.rating from ratings r, movies m where m.movieID = r.movieID order by r.rating desc limit 5"
spark.sql(query).show()

+--------------------+--------------------+------+
|               title|              genres|rating|
+--------------------+--------------------+------+
|       Freaks (1932)|  Crime|Drama|Horror|   5.0|
|Legends of the Fa...|Drama|Romance|War...|   5.0|
|Mr. Holland's Opu...|               Drama|   5.0|
|Lord of the Rings...|   Adventure|Fantasy|   5.0|
|From Dusk Till Da...|Action|Comedy|Hor...|   5.0|
+--------------------+--------------------+------+



### Query 10: Which are the top rated by the users Sci-fi movies?

In [24]:
query = "Select m.title,m.genres as Genre_SCIFI,r.rating from movies m,ratings r where genres Like '%Sci-Fi%' and m.movieID = r.movieID order by r.rating desc"
spark.sql(query).show()

+--------------------+--------------------+------+
|               title|         Genre_SCIFI|rating|
+--------------------+--------------------+------+
|Ghostbusters (a.k...|Action|Comedy|Sci-Fi|   5.0|
|Jurassic Park (1993)|Action|Adventure|...|   5.0|
| Donnie Darko (2001)|Drama|Mystery|Sci...|   5.0|
|Independence Day ...|Action|Adventure|...|   5.0|
|        Signs (2002)|Horror|Sci-Fi|Thr...|   5.0|
| Blade Runner (1982)|Action|Sci-Fi|Thr...|   5.0|
| Dreamcatcher (2003)|Drama|Horror|Sci-...|   5.0|
|City of Lost Chil...|Adventure|Drama|F...|   5.0|
|Interstate 60 (2002)|Adventure|Comedy|...|   5.0|
|Star Wars: Episod...|Action|Adventure|...|   5.0|
|The Butterfly Eff...|Drama|Sci-Fi|Thri...|   5.0|
|        Alien (1979)|       Horror|Sci-Fi|   5.0|
|Eternal Sunshine ...|Drama|Romance|Sci-Fi|   5.0|
|Truman Show, The ...| Comedy|Drama|Sci-Fi|   5.0|
|Battlestar Galact...|    Drama|Sci-Fi|War|   5.0|
|Terminator 2: Jud...|       Action|Sci-Fi|   5.0|
|     Serenity (2005)|Action|Ad

### Query 11: Which are the Worst Comedy movies rated by the users in the year 2012?

In [25]:
query = "Select distinct(m.title),m1.title as Year,m.genres as Genre_Comedy,r.rating from movies m, ratings r,movies_year as m1 where m.movieID = r.movieID and m.movieID = m1.movieID and r.movieID = m1.movieId and m.genres Like '%Comedy%' and m1.title = '2012' order by r.rating"
spark.sql(query).show()

+--------------------+----+--------------------+------+
|               title|Year|        Genre_Comedy|rating|
+--------------------+----+--------------------+------+
|Thanks for Sharin...|2012|        Comedy|Drama|   0.5|
|John Mulaney: New...|2012|              Comedy|   0.5|
|Home Alone: The H...|2012|Children|Comedy|C...|   0.5|
|Much Ado About No...|2012|Comedy|Drama|Romance|   0.5|
|21 Jump Street (2...|2012| Action|Comedy|Crime|   0.5|
|Excuse Me for Liv...|2012|      Comedy|Romance|   0.5|
| Bachelorette (2012)|2012|              Comedy|   0.5|
|        Vamps (2012)|2012|Comedy|Horror|Rom...|   0.5|
| Rock of Ages (2012)|2012|Comedy|Drama|Musi...|   0.5|
|Christmas Story 2...|2012|     Children|Comedy|   0.5|
|3, 2, 1... Franki...|2012|              Comedy|   0.5|
|Five-Year Engagem...|2012|      Comedy|Romance|   0.5|
|2-Headed Shark At...|2012|       Comedy|Horror|   0.5|
| Tai Chi Hero (2012)|2012|Action|Comedy|Dra...|   0.5|
|Parental Guidance...|2012|              Comedy|